In [1]:
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapflyError
from jsonpath_ng import jsonpath, parse
 
scrapfly = ScrapflyClient(key="scp-live-01fc8534987042f4a2fe553c5cf6a2df", max_concurrency=2)
url = 'https://doddcamera.com/dji267801'
listing_result = scrapfly.scrape(ScrapeConfig(url=url, render_js=True, country="US", asp=True, retry=False, rendering_wait=10000))
listing_html = listing_result.scrape_result['content']

In [13]:
!pip install jsonpath-ng

  Obtaining dependency information for jsonpath-ng from https://files.pythonhosted.org/packages/16/0a/3b1ee3721b4bd684b0e29c724ab82ed3b2c0e42285beb8bf9e18de8c903f/jsonpath_ng-1.6.1-py3-none-any.whl.metadata


In [2]:
from bs4 import BeautifulSoup
import re
import json
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

json_data_list = extract_json_from_html(listing_html)

In [16]:
from jsonpath_ng import jsonpath, parse
paths = {
    'name': "$..name",
    'url': "$..baseUrl",
    'id': "$..productId",
    'currency': "$..currency",
    'price_info_final_price': "$..price_info.final_price",  # Example for price
    'country_code': "$..country_code",
    'type': "$..productType",
    'images': "$..images",
    'productcode': "$..productId"
}

In [17]:
extracted_data = {}
for key, path in paths.items():
    jsonpath_expr = parse(path)
    matches = jsonpath_expr.find(json_data_list)
    extracted_data[key] = [match.value for match in matches]

In [18]:
extracted_data

{'name': ['DJI Mavic 3 Classic Fly More Kit'],
 'url': ['https://doddcamera.com/', 'https://doddcamera.com/'],
 'id': ['21217'],
 'currency': ['USD', 'USD'],
 'price_info_final_price': [649],
 'country_code': ['USA'],
 'type': ['simple'],
 'images': [[{'url': 'https://doddcamera.com/media/catalog/product/cache/7c0512946b1fb1ced7b27d67c72a822f/d/j/dji_mavic_3-_fly_more_kit-1_1.jpg',
    'code': 'recently_viewed_products_grid_content_widget',
    'height': 335,
    'width': 270,
    'label': 'DJI Mavic 3 Classic Fly More Kit',
    'resized_width': 270,
    'resized_height': 335},
   {'url': 'https://doddcamera.com/media/catalog/product/cache/247c45a58ca593a31e941c9930dbd355/d/j/dji_mavic_3-_fly_more_kit-1_1.jpg',
    'code': 'recently_viewed_products_list_content_widget',
    'height': 300,
    'width': 300,
    'label': 'DJI Mavic 3 Classic Fly More Kit',
    'resized_width': 300,
    'resized_height': 300},
   {'url': 'https://doddcamera.com/media/catalog/product/cache/c045ef02d07a7328